In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

In [2]:
#Load ESCAPE training data
escapeAllData = pd.read_csv("Data/Original DataFrames/AllDataSingleValue.csv", sep=",", index_col='DEIDNUM').sort_index() #all feature dataset
escapeHemo = pd.read_csv("Data/Original DataFrames/HemoSingleValue.csv", sep=",", index_col='DEIDNUM').sort_index() #dataset with only hemodynamics
escapeLabels  = pd.read_csv("Data/Original DataFrames/Labels.csv", sep=",", index_col='DEIDNUM').sort_index() #labels for prediction classes 

#Cardiogenic Shock
cardShockHemo = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/HemoCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
cardShockAllData = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/AllDataCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
cardShockHemoLabels = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/HemoLabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
cardShockLabels = pd.read_csv("Data Validation/Cardiogenic Shock/Original DataFrames/LabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()

#Serial Cardiac
serialHemo = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/HemoSerialCardiac.csv", sep=",", index_col='ID').sort_index()
serialAllData = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/AllDataSerialCardiac.csv", sep=",", index_col='ID').sort_index()
serialHemoLabels = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/HemoLabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()
serialLabels = pd.read_csv("Data Validation/Serial Cardiac Caths/Original DataFrames/LabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()

#HF-ACTION
hfactionAllData = pd.read_csv("Data Validation/HF-ACTION/Original DataFrames/AllDataHF-ACTION.csv", sep=",", index_col='ID').sort_index()
hfactionLabels = pd.read_csv("Data Validation/HF-ACTION/Original DataFrames/LabelsHF-ACTION.csv", sep=",", index_col='ID').sort_index()

#BEST
bestAllData = pd.read_csv("Data Validation/BEST/Original DataFrames/AllDataBEST.csv", sep=",", index_col='ID').sort_index()
bestLabels = pd.read_csv("Data Validation/BEST/Original DataFrames/LabelsBEST.csv", sep=",", index_col='ID').sort_index()

#Guide it
guideAllData = pd.read_csv("Data Validation/GUIDE-IT/Original DataFrames/AllDataGUIDE-IT.csv", sep=",", index_col='ID').sort_index()
guideLabels = pd.read_csv("Data Validation/GUIDE-IT/Original DataFrames/LabelsGUIDE-IT.csv", sep=",", index_col='ID').sort_index()


In [3]:
escapeHemo.describe()

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
count,418.000000,418.000000,418.000000,404.000000,376.000000,369.000000,369.000000,378.000000,351.000000,365.000000,...,377.000000,377.000000,369.000000,355.000000,344.000000,377.000000,362.000000,345.000000,380.000000,369.000000
mean,56.789474,1.263158,3.885167,19.400990,11.686170,50.867209,24.298103,33.730159,21.404558,4.299151,...,43.954907,0.409138,0.523738,2852.094746,inf,0.606385,inf,inf,40.678947,inf
std,13.492824,0.440875,14.779473,6.841169,8.724061,14.290132,8.824867,11.106248,8.936371,1.768310,...,16.354471,0.144257,0.101996,1222.825988,NaN,0.651917,NaN,NaN,13.793313,NaN
min,23.000000,1.000000,1.000000,0.000000,0.000000,20.000000,5.000000,1.000000,0.000000,1.200000,...,-55.000000,-0.833333,0.230769,381.866667,0.000000,-0.887097,0.433333,-1.851852,10.000000,-8.333333
25%,48.000000,1.000000,1.000000,15.000000,6.000000,40.000000,18.000000,26.000000,15.000000,3.250000,...,35.000000,0.345238,0.444444,2050.122585,0.333333,0.406977,1.666667,1.400000,32.000000,2.550000
50%,57.000000,1.000000,1.000000,20.000000,10.000000,50.000000,24.000000,33.000000,20.000000,4.100000,...,43.000000,0.418182,0.519231,2620.952381,0.500000,0.537500,2.641711,2.166667,38.000000,4.500000
75%,66.000000,2.000000,2.000000,25.000000,15.250000,60.000000,29.000000,40.000000,27.000000,5.100000,...,52.000000,0.477778,0.595238,3277.893970,0.670000,0.720000,4.000000,3.333333,50.000000,7.500000
max,88.000000,2.000000,98.000000,45.000000,85.000000,90.000000,59.000000,82.000000,53.000000,25.000000,...,106.000000,0.736842,0.844444,10755.555556,inf,9.666667,inf,inf,83.000000,inf


In [4]:
def getStats(data, labels):
    if len(data) == len(labels) * 2:
        print("Age:", str(round(data.describe()['Age'][1],1)) + "[" + str(round(data.describe()['Age'][2],1)) + "]")
        gen = data['Gender'].value_counts()/2/len(labels)
        print("Gender", round(gen[2.0],3))
        gen = data['Race'].value_counts()/2/len(labels)
        print("Race", str(round(gen[1.0],3))+"[" + str(round(gen[2.0],3))+"]")
    else:
        print("Age:", str(round(data.describe()['Age'][1],1)) + "[" + str(round(data.describe()['Age'][2],1)) + "]")
        gen = data['Gender'].value_counts()/len(labels)
        print("Gender", round(gen[2.0],3))
        gen = data['Race'].value_counts()/len(labels)
        print("Race", str(round(gen[1.0],3))+"[" + str(round(gen[2.0],3))+"]")
    
getStats(escapeAllData, escapeLabels)

Age: 56.1[13.9]
Gender 0.259
Race 0.596[0.404]


In [5]:
getStats(hfactionAllData, hfactionLabels)

Age: 58.6[12.7]
Gender 0.281
Race 0.621[0.379]


In [6]:
getStats(bestAllData, bestLabels)

Age: 60.2[12.3]
Gender 0.219
Race 0.7[0.3]


In [7]:
getStats(guideAllData, guideLabels)

Age: 61.5[13.9]
Gender 0.68
Race 0.548[0.452]


In [8]:
def getStatsCard(data, labels):
    print("Age:", str(round(data.describe()['Age'][1],1)) + "[" + str(round(data.describe()['Age'][2],1)) + "]")
    gen = data['Gender'].value_counts()/3/len(labels)
    print("Gender", round(gen[2.0],3))



In [9]:
getStatsCard(cardShockAllData, cardShockAllData)

Age: 59.4[18.5]
Gender 0.117


In [10]:
getStatsCard(serialAllData, serialLabels)

Age: 60.6[15.1]
Gender 0.432
